In [ ]:
!pip install transformers datasets torch pandas sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+c

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

# Load datasets (assuming they are uploaded to Colab or stored in Google Drive)
# If in Google Drive, mount it:
# from google.colab import drive
# drive.mount('/content/drive')

fake_df = pd.read_csv('/content/drive/MyDrive/ma_fake.csv')  # Adjust path as needed
true_df = pd.read_csv('/content/drive/MyDrive/ma_true.csv')  # Adjust path as needed

# Add labels
fake_df['label'] = 0  # Fake news
true_df['label'] = 1  # True news

# Combine datasets
df = pd.concat([fake_df, true_df], ignore_index=True)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Basic preprocessing for Malayalam text
def preprocess_text(text):
    # Remove special characters, numbers, and extra spaces (adjust as needed for Malayalam)
    text = str(text).strip()
    text = ''.join(char for char in text if char.isalpha() or char.isspace() or char in 'അആഇഈഉഊഋഎഏഐഒഓൺം')
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/6238 [00:00<?, ? examples/s]

Map:   0%|          | 0/1560 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers

In [ ]:
# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Define training arguments
training_args = TrainingArguments( # Changed 'raining_args' to 'training_args'
    output_dir='./results',
    num_train_epochs=5,  # Increase epochs
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=200,  # Reduce warmup steps if dataset is small
    weight_decay=0.1,  # Increase regularization
    learning_rate=2e-5,  # Smaller learning rate for better convergence
    logging_dir='./logs',
    logging_steps=10,
    # evaluation_strategy="epoch", # Removed: This may be causing issues with your version of transformers
    # save_strategy="epoch", # Removed: Check for alternative saving methods in the documentation
    # load_best_model_at_end=True, # Removed: Check if this is still supported or has been renamed
    # Instead of these, you might consider:
    save_total_limit=1,  # Only save the best checkpoint
    eval_strategy = "steps", # or "epoch" , Check if "steps" works with your version
    eval_steps = 100 # or adjust this for your needs
)

# Define compute_metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayadev262122 (jayadev262122-sreepathy-institute-of-management-and-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.462700,0.430208,0.816026,0.828338,0.790637,0.809049
200,0.529400,0.605979,0.798077,0.939922,0.630689,0.754864
300,0.331800,0.262214,0.905769,0.868483,0.953186,0.908865
400,0.234400,0.298201,0.909615,0.889578,0.932380,0.910476
500,0.405600,0.258810,0.926923,0.907846,0.947984,0.927481
600,0.352500,0.275485,0.928205,0.971306,0.880364,0.923602
700,0.336100,0.202498,0.942949,0.919753,0.968791,0.943635
800,0.158700,0.200977,0.954487,0.946292,0.962289,0.954223
900,0.075400,0.218132,0.944231,0.965847,0.919376,0.942039
1000,0.167900,0.167126,0.953205,0.927518,0.981795,0.953885


TrainOutput(global_step=3900, training_loss=0.1676365983262897, metrics={'train_runtime': 1513.8879, 'train_samples_per_second': 20.603, 'train_steps_per_second': 2.576, 'total_flos': 2051608454169600.0, 'train_loss': 0.1676365983262897, 'epoch': 5.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.18059970438480377, 'eval_accuracy': 0.967948717948718, 'eval_precision': 0.9687092568448501, 'eval_recall': 0.9661898569570871, 'eval_f1': 0.9674479166666666, 'eval_runtime': 11.9886, 'eval_samples_per_second': 130.124, 'eval_steps_per_second': 16.265, 'epoch': 5.0}


In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/lastdance')
tokenizer.save_pretrained('/content/drive/MyDrive/lastdance')

('/content/drive/MyDrive/lastdance/tokenizer_config.json',
 '/content/drive/MyDrive/lastdance/special_tokens_map.json',
 '/content/drive/MyDrive/lastdance/vocab.txt',
 '/content/drive/MyDrive/lastdance/added_tokens.json')

In [ ]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/lastdance')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/lastdance')

# Function to predict on new text
def predict_fake_news(text):
    # Preprocess and tokenize the text
    text = preprocess_text(text)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    return "True" if prediction == 1 else "Fake"
# Example usage
new_text = "2000 രൂപ നോട്ടിന്റെ വിതരണം നിര്‍ത്തി റിസര്‍വ് ബാങ്ക്"  # Replace with your Malayalam text
result = predict_fake_news(new_text)
print(f"The text is predicted to be: {result}")
# Example usage
new_text = "'യുഡിഎഫ് കള്ളം പ്രചരിപ്പിക്കാറില്ല, ഉള്ളത് മാത്രമേ പറയു' പിണറായി വിജയൻ ഇങ്ങനെ പറഞ്ഞോ?..."  # Replace with your Malayalam text
result = predict_fake_news(new_text)
print(f"The text is predicted to be: {result}")

The text is predicted to be: True
The text is predicted to be: Fake


In [ ]:
from sklearn.metrics import classification_report

# Get predictions on the test set
predictions = trainer.predict(test_dataset)

# Extract labels and predictions
labels = predictions.label_ids
preds = predictions.predictions.argmax(-1)

# Print detailed classification report
print("Classification Report:")
print(classification_report(labels, preds, target_names=['True', 'Fake']))

Classification Report:
              precision    recall  f1-score   support

        True       0.97      0.97      0.97       791
        Fake       0.97      0.97      0.97       769

    accuracy                           0.97      1560
   macro avg       0.97      0.97      0.97      1560
weighted avg       0.97      0.97      0.97      1560



In [ ]:
def predict_fake_news(text):
    text = preprocess_text(text)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
    return "True" if prediction == 1 else "Fake"  # Swap if necessary

In [ ]:
# Example usage
new_text = "മോദി ലോകത്തിൻ്റെ പ്രതീക്ഷ  പ്രധാനമന്ത്രിയെ പുകഴ്ത്തിയ ന്യൂ യോർക്ക് ടൈംസ് വാർത്ത വ്യാജമോ?"  # Replace with your Malayalam text
result = predict_fake_news(new_text)
print(f"The text is predicted to be: {result}")

The text is predicted to be: Fake


In [ ]:
# Example usage
new_text = "പത്മഭൂഷണ്‍ പുരസ്‌കാരം ഏറ്റുവാങ്ങി മോഹന്‍ലാല്‍."  # Replace with your Malayalam text
result = predict_fake_news(new_text)
print(f"The text is predicted to be: {result}")

The text is predicted to be: True


In [ ]:
!pip install gradio
import gradio as gr

# Define the improved UI function
def fake_news_checker(text):
    result = predict_fake_news(text)  # Ensure predict_fake_news() returns a clean string
    return result  # Directly return the result without adding extra symbols

# Design the webpage with Gradio Blocks
with gr.Blocks(theme=gr.themes.Soft(), css="body {background: linear-gradient(to right, #1E3C72, #2A5298);}") as demo:
    with gr.Row():
        gr.Markdown("📰 Fake News Detector")

    with gr.Row():
        with gr.Column(scale=2):
            text_input = gr.Textbox(placeholder="Enter the news title here...", label="News Title", lines=2)
        with gr.Column(scale=1):
            predict_button = gr.Button("🔍 Check", variant="primary")

    with gr.Row():
        output = gr.Textbox(label="Prediction", interactive=False)

    predict_button.click(fn=fake_news_checker, inputs=text_input, outputs=output)

# Launch the app
demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dac46d7cf47ce6abec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
